In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow
import string
import seaborn as sns
import matplotlib.pyplot as plt
import re
# VISUALIZATION
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

In [2]:
pd.read_csv('/kaggle/input/yelp-reviews/test.csv')

,Polarity,Comments,Unnamed: 2
0,2,"Contrary to other reviews, I have zero complai...",NaN
1,1,Last summer I had an appointment to get new ti...,NaN
2,2,"Friendly staff, same starbucks fair you get an...",NaN
3,1,The food is good. Unfortunately the service is...,NaN
4,2,Even when we didn't have a car Filene's Baseme...,NaN
...,...,...,...
37995,1,If I could give 0...I would. Don't do it.,NaN
37996,2,Items Selected:\nChocolate Cinnamon Horn\nSmal...,NaN
37997,1,Expensive lunch meals. Fried pickles were goo...,NaN
37998,1,Highly overpriced and food was cold. Our waitr...,NaN


In [3]:
train = pd.read_csv('/kaggle/input/yelp-reviews/train.csv')
#test = pd.read_csv('/kaggle/input/test.csv')

In [4]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have","sbse":"biggest","chor":"thief","paisa":"money","barbaad":"waste",
                            "grmi":"summer","chori":"steal","paise":"money","sbse bada":"biggest","bada":"big","❤":"love"}

In [5]:
#test["Polarity"]=test["Polarity"].astype("str")
#test["Polarity"]=test["Polarity"].str.replace("1","negative")
#test["Polarity"]=test["Polarity"].str.replace("2","positive")

train["Polarity"]=train["Polarity"].astype("str")
train["Polarity"]=train["Polarity"].str.replace("1","negative")
train["Polarity"]=train["Polarity"].str.replace("2","positive")
labels = train["Polarity"].values.tolist()

In [6]:
def process(sen):
    processed = []
    for i in range(0, len(sen)):
        processed_feature = re.sub(r'\([^)]*\)', '', str(sen[i]))
        processed_feature = re.sub('"','', processed_feature)
        processed_feature= ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in processed_feature.split(" ")])
        
    # Converting to Lowercase
        processed_feature = processed_feature.lower()
        
        processed_feature=re.sub(r'^https?:\/\/.*[\r\n]*', '', processed_feature, flags=re.MULTILINE)
        
    # Remove all the special characters
        
        #processed_feature = re.sub(r'\W', ' ', processed_feature)

    # remove all single characters
        processed_feature= re.sub("[^a-z0-9❤, ]", "", processed_feature)
        processed_feature= re.sub(r'\s+[a-z]\s+', ' ', processed_feature)
        

    # Substituting multiple spaces with single space
        processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
        processed_feature = re.sub(r'^b\s+', '', processed_feature)


        processed.append(processed_feature.strip())
    return processed

In [7]:
train[:100000]

,Polarity,Comments
0,negative,"Unfortunately, the frustration of being Dr. Go..."
1,positive,Been going to Dr. Goldberg for over 10 years. ...
2,negative,I don't know what Dr. Goldberg was like before...
3,negative,I'm writing this review to give you a heads up...
4,positive,All the food is great here. But the best thing...
...,...,...
99995,positive,After my final walk out of Quick Wok on Southe...
99996,positive,"Sadly, I went to pick up my yummy food last Fr..."
99997,negative,Been going here since they open on the corner ...
99998,negative,I was really excited to try out Tott's after h...


In [8]:
process_train=process(train["Comments"][:200000])

In [9]:
#process_test=process(test["Comments"])

In [10]:
input_sentences = [text.split(" ") for text in process_train]

In [11]:
#Creating Vocabulary (word index)
# Initialize word2id and label2id dictionaries that will be used to encode words and labels
word2id = dict()
label2id = dict()

max_words = 0 # maximum number of words in a sentence

# Construction of word2id dict
for sentence in input_sentences:
    for word in sentence:
        # Add words to word2id dict if not exist
        if word not in word2id:
            word2id[word] = len(word2id)
    # If length of the sentence is greater than max_words, update max_words
    if len(sentence) > max_words:
        max_words = len(sentence)
    
# Construction of label2id and id2label dicts
label2id = {j: i for i, j in enumerate(set(labels))}
id2label = {v: k for k, v in label2id.items()}
id2label
label2id

{'positive': 0, 'negative': 1}

In [12]:
# Encode input words and labels
X=[]
Y=[]

X = [[word2id[word] for word in sentence] for sentence in input_sentences]
    
Y = [label2id[label] for label in labels[:200000]]
X

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  3,
  1,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  23,
  28,
  29,
  30,
  1,
  31,
  24,
  32,
  33,
  34,
  35,
  3,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  26,
  46,
  47,
  38,
  48,
  13,
  24,
  11,
  49,
  50,
  51,
  52,
  13,
  15,
  16,
  17,
  53,
  54,
  55,
  56,
  39,
  24,
  57,
  11,
  58,
  59,
  57,
  11,
  60,
  13,
  61,
  62,
  63,
  8,
  56,
  64,
  65,
  1,
  31,
  66,
  67,
  53,
  56,
  68,
  1,
  69,
  66,
  13,
  70,
  26,
  71,
  26,
  11,
  38,
  72,
  5,
  73,
  34,
  74],
 [75,
  76,
  38,
  5,
  73,
  45,
  77,
  78,
  79,
  80,
  81,
  82,
  3,
  27,
  83,
  60,
  84,
  85,
  86,
  87,
  88,
  89,
  75,
  90,
  77,
  1,
  79,
  53,
  8,
  91,
  92,
  93,
  1,
  94,
  95,
  24,
  8,
  96,
  3,
  97,
  56,
  98,
  99,
  100,
  101,
  5,
  102,
  26,
  103,
  104,
  11,
  105,
  85,
  106,
  92,
  107,
  13,
  57,
  5

In [13]:
# Apply Padding to X
import keras
from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, max_words)

# Convert Y to numpy array
Y = keras.utils.to_categorical(Y, num_classes=len(label2id), dtype='float32')

# Print shapes
print("Shape of X: {}".format(X.shape))
print("Shape of Y: {}".format(Y.shape))

Shape of X: (200000, 960)
Shape of Y: (200000, 2)


In [14]:
max_words

960

In [15]:
embedding_dim = 100 # The dimension of word embeddings

# Define input tensor
sequence_input = keras.Input(shape=(max_words,), dtype='int32')

# Word embedding layer
embedded_inputs =keras.layers.Embedding(len(word2id) + 1,
                                        embedding_dim,
                                        input_length=max_words)(sequence_input)

# Apply dropout to prevent overfitting
embedded_inputs = keras.layers.Dropout(0.2)(embedded_inputs)

# Apply Bidirectional LSTM over embedded inputs
lstm_outs = keras.layers.wrappers.Bidirectional(
    keras.layers.LSTM(embedding_dim, return_sequences=True)
)(embedded_inputs)

# Apply dropout to LSTM outputs to prevent overfitting
lstm_outs = keras.layers.Dropout(0.2)(lstm_outs)

# Attention Mechanism - Generate attention vectors
input_dim = int(lstm_outs.shape[2])
permuted_inputs = keras.layers.Permute((2, 1))(lstm_outs)
attention_vector = keras.layers.TimeDistributed(keras.layers.Dense(1))(lstm_outs)
attention_vector = keras.layers.Reshape((max_words,))(attention_vector)
attention_vector = keras.layers.Activation('softmax', name='attention_vec')(attention_vector)
attention_output = keras.layers.Dot(axes=1)([lstm_outs, attention_vector])

# Last layer: fully connected with softmax activation
fc = keras.layers.Dense(embedding_dim, activation='relu')(attention_output)
output = keras.layers.Dense(len(label2id), activation='softmax')(fc)

# Finally building model
model = keras.Model(inputs=[sequence_input], outputs=output)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam')

# Print model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 960)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 960, 100)     36700800    input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 960, 100)     0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 960, 200)     160800      dropout[0][0]                    
______________________________________________________________________________________________

In [16]:
# Train model 2 iterations
model.fit(X, Y, epochs=2, batch_size=512, validation_split=0.20, shuffle=True)

Epoch 1/2
313/313 [==============================] - 236s 754ms/step - loss: 0.3271 - accuracy: 0.8390 - val_loss: 0.2069 - val_accuracy: 0.9157
Epoch 2/2
313/313 [==============================] - 231s 738ms/step - loss: 0.1458 - accuracy: 0.9444 - val_loss: 0.1955 - val_accuracy: 0.9237


In [17]:
# Re-create the model to get attention vectors as well as label prediction
model_with_attentions = keras.Model(inputs=model.input, 
                                    outputs=[model.output, 
                                             model.get_layer('attention_vec').output])

In [18]:
!pip install googletrans

     |████████████████████████████████| 55 kB 2.0 MB/s eta 0:00:011
     |████████████████████████████████| 42 kB 864 kB/s  eta 0:00:01
     |████████████████████████████████| 917 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 2.9 MB/s  eta 0:00:01
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15734 sha256=ef18f0b95ec24109ff73f3c9125a658c81d50cc8cb65126ed36f7e873e435e88
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans


In [19]:
from googletrans import Translator
translator = Translator(service_urls=['translate.google.co.in'])


def translate(word):
	return translator.translate(word,src='hi' , dest='en')

In [20]:
import random
import math

In [21]:
#sample_text='Sbse bada chor Quikr paise ceo "Pranay" usko email kro to b koi rply ni ata kyu paisa to ussi ki jeb me jata h.. I hope Quikr abi bankruptcy ni h aur tmko notice mil gya hoga meri side se'
#len(sample_text.split())

In [22]:
#translate(sample_text[:20]).text

In [23]:

from nltk.tokenize import sent_tokenize,word_tokenize
  
#text = "Hello everyone. Welcome to GeeksforGeeks. You are studying NLP article"
#sent_tokenize(text) 

In [24]:
#translate(sent_tokenize(sample_text)[0]).text 

In [26]:
df=pd.read_excel('/kaggle/input/companies/hometriangle.xlsx')
#quikr=pd.read_excel('/kaggle/input/companies/quikr_official.xlsx')urbancompany.xlsx

In [27]:
df["comments"]=df["comments"].replace("[]","")
#quikr["comments"]=quikr["comments"].replace("[]","")

In [28]:
com=list(df["comments"])

In [29]:
for i in range(len(com)):
    if com[i]!="":
        com[i]=com[i][1:len(com[i])-1]
        com[i]=com[i].split("',")

In [31]:
df["comments"].tolist()

['',
 "['Have you done your website Search Engine Optimization (SEO) with in free trial during these lockdown days??']",
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 "['👍']",
 '',
 '',
 '',
 '',
 '["Happy mother\'s day"]',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 "['👌🏻👌🏻👌🏻', 'much required...']",
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 "['👍 Tips', 'Liked this']",
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 "['Happy holi']",
 '',
 "['Happy Holi', 'Happy holi 😊']",
 '',
 '',
 '',
 '',
 '',
 "['I almost give up on ever getting cured of HSV2 because I have try many treatment none of them work out for me I have go to different hospital they always tell me same thing there is no cure for herpes when I come across a post about @dr_agbon in the net from a lady called Angela I contacted him and he reassured me with him herb

In [32]:
#processed_com=[]
for i in range(len(com)):
    if com[i]!="":
        com[i]=process(com[i])
    

In [33]:
new=[]
for i in range(len(com)):
    if com[i]!="" and com[i]!=[]:
        new1=[]
        for j in range(len(com[i])):
            if com[i][j]!="":
                com[i][j]=translate(com[i][j]).text
                #s=""
                #for word in com[i][j].split():
                #    s+=translate(word).text+" "
                new1.append(com[i][j].strip())
            else:
                new1.append("")
        new.append(new1)
    else:
        new.append("")

In [34]:
for i in range(len(new)):
    if new[i]!="":
        if "" in new[i]:
            new[i].remove("") 

In [35]:
# Encode samples
preds=[]
for i in range(len(new)):
    
    if new[i]=='' or new[i]==[]:
        preds.append("No comments")
    else:
        pred_score=[]
        #emo=[]
        for j in range(len(new[i])):
            
            tokenized_sample = new[i][j].split(" ")
            encoded_samples = [[word2id[word] for word in tokenized_sample if word in word2id]]
            encoded_samples = keras.preprocessing.sequence.pad_sequences(encoded_samples, maxlen=max_words)
               
            label_probs, attentions = model_with_attentions.predict(encoded_samples)
            label_probs = {id2label[_id]: prob for (label, _id), prob in zip(label2id.items(),label_probs[0])}
            pred_score.append(label_probs)
            
        preds.append(pred_score) 
                 
   

In [36]:
#translate(new[4][1]).text

In [37]:
new

['',
 ['have you done your website search engine optimization with in free trial during these lockdown days'],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 [],
 '',
 '',
 '',
 '',
 ['happy mothers day'],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ['much required'],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ['tips', 'liked this'],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ['happy holi'],
 '',
 ['happy holi', 'happy holi'],
 '',
 '',
 '',
 '',
 '',
 ['i almost give up on ever getting cured of hsv2 because have try many treatment none of them work out for me have go to different hospital they always tell me the same thing there is no cure for herpes when come across post about dragbon in the net from lady called angela argued him and he reassured me with him herbal medicine which took according to the way he instructed,

In [38]:
preds

['No comments',
 [{'positive': 0.50854975, 'negative': 0.4914503}],
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 [{'positive': 0.96480954, 'negative': 0.035190407}],
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 [{'positive': 0.5581148, 'negative': 0.4418851}],
 'No comments',
 'No comments',
 'No commen

In [39]:
overall=[]
pos_total=[]
neg_total=[]
total_pol=[]
for i in range(len(preds)):
    if preds[i]!="No comments":
        for emo in preds[i]:
            pos=0.0
            neg=0.0
            for k, v in emo.items():
                print('{}: {}'.format(k, v))
                if str(k)=='positive':
                    pos+=v
                    
                else:
                    neg+=v
        if pos>0.6:
            overall.append("positive")
        elif neg>0.6:
            overall.append("negative")
        else:
            overall.append("neutral")
            
        pos_total.append(pos)
        neg_total.append(neg)
        total_pol.append(pos-neg)
    else:
        pos_total.append("No comments")
        neg_total.append("No comments")
        overall.append("No comments")
        total_pol.append("No comments")

positive: 0.5085497498512268
negative: 0.49145030975341797
positive: 0.9648095369338989
negative: 0.0351904071867466
positive: 0.5581148266792297
negative: 0.4418851137161255
positive: 0.7255553603172302
negative: 0.2744446098804474
positive: 0.9259100556373596
negative: 0.07409000396728516
positive: 0.9831545352935791
negative: 0.01684543490409851
positive: 0.9831545352935791
negative: 0.01684543490409851
positive: 0.9831545352935791
negative: 0.01684543490409851
positive: 0.9163498878479004
negative: 0.08365003764629364
positive: 0.9574647545814514
negative: 0.04253528267145157
positive: 0.38622617721557617
negative: 0.6137738227844238
positive: 0.38622617721557617
negative: 0.6137738227844238
positive: 0.9652673006057739
negative: 0.034732673317193985
positive: 0.573100209236145
negative: 0.426899790763855
positive: 0.9970383644104004
negative: 0.0029616893734782934
positive: 0.9765507578849792
negative: 0.023449210450053215
positive: 0.9267051219940186
negative: 0.07329489290714264

In [52]:
pos_total[110]

0.38622617721557617

In [53]:
neg_total[110]

0.6137738227844238

In [42]:
overall

['No comments',
 'neutral',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'positive',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'neutral',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'N

In [43]:
preds

['No comments',
 [{'positive': 0.50854975, 'negative': 0.4914503}],
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 [{'positive': 0.96480954, 'negative': 0.035190407}],
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 [{'positive': 0.5581148, 'negative': 0.4418851}],
 'No comments',
 'No comments',
 'No commen

In [44]:
total_pol

['No comments',
 0.017099440097808838,
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 0.9296191297471523,
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 0.11622971296310425,
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No comments',
 'No c

In [45]:
df["total_polarity"]=total_pol
df["overall"]=overall

In [46]:
df.to_excel("hometriangle_sentiment.xlsx")